In [68]:
# Problem 3 BB84 Protocol
from qiskit import *
from qiskit.visualization import plot_histogram
from numpy.random import randint
import numpy as np
from math import pi
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute, BasicAer, IBMQ
from qiskit import QuantumCircuit, execute, Aer

In [69]:
n = 100; # number of qubits used in the BB84
np.random.seed(seed=3)
## Step 1
# Alice generates bits
alice_bits = randint(2, size=n)
print(alice_bits)
print("Alice's first bit = %i" % alice_bits[0])

[0 0 1 1 0 0 0 1 1 1 0 1 1 1 0 1 1 0 0 0 0 1 1 0 0 0 1 0 0 0 0 1 0 1 1 0 1
 0 0 1 1 0 0 1 0 1 0 1 1 1 1 0 1 0 0 1 1 1 0 0 0 1 0 0 0 1 0 0 1 1 0 0 1 1
 1 0 1 1 1 1 1 1 0 1 0 0 1 1 0 1 0 0 0 0 0 1 1 0 1 1]
Alice's first bit = 0


In [70]:
def encode_message(bits, bases):
    message = []
    for i in range(n):
        qc = QuantumCircuit(1,1)
        if bases[i] == 0: # Prepare qubit in X-basis
            if bits[i] == 0:
                pass 
            else:
                qc.x(0)
        else: # Prepare qubit in Z-basis
            if bits[i] == 0:
                qc.h(0)
            else:
                qc.x(0)
                qc.h(0)
        qc.barrier()
        message.append(qc)
    return message

In [71]:
## Step 2
# Create an array to tell us which qubits are encoded in which bases
alice_bases = randint(2, size=n)
message = encode_message(alice_bits, alice_bases)
print(alice_bases)
print("Alice's first chosen bit = %i" % alice_bits[0])
print("Alice's first chosen basis = %i" % alice_bases[0])

[1 0 0 1 1 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1 1 1 0 0 1 1 0 0 1 1 1 1 0 0
 1 1 0 0 0 0 0 0 0 0 1 1 0 0 0 1 0 1 1 1 0 0 1 0 1 0 1 1 0 0 1 0 0 1 1 1 1
 1 0 0 0 0 0 1 1 1 0 0 0 0 0 0 1 0 1 0 0 0 1 0 1 1 1]
Alice's first chosen bit = 0
Alice's first chosen basis = 1


In [72]:
def intercept_message(message, bases):
    backend = Aer.get_backend('qasm_simulator')
    measurements = []
    for q in range(n):
        if bases[q] == 0: # measuring in Z-basis
            message[q].measure(0,0)
        if bases[q] == 1: # measuring in X-basis
            message[q].h(0)
            message[q].measure(0,0)
            message[q].h(0) # preparing the post-measurement state
        result = execute(message[q], backend, shots=1, memory=True).result()
        measured_bit = int(result.get_memory()[0])
        measurements.append(measured_bit)
    return measurements

In [73]:
## Interception!!
eve_bases = randint(2, size=n)
intercepted_message = intercept_message(message, eve_bases)
print(eve_bases)
print("Eve's first chosen basis = %i\n" % alice_bases[0])
print(intercepted_message)
print("Eve's first intercepted message = %i" % intercepted_message[0])

[0 1 1 0 1 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 1 0 1 0 1 0 1 0 1 0 1 1 0 1
 0 1 0 1 0 1 1 1 1 1 0 1 0 1 1 1 1 0 0 0 1 0 1 0 0 1 0 1 0 0 0 1 0 0 1 1 1
 0 1 0 1 0 1 1 0 1 1 0 1 1 0 0 0 1 1 1 0 1 0 0 0 1 0]
Eve's first chosen basis = 1

[1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1]
Eve's first intercepted message = 1


In [74]:
def measure_message(message, bases):
    backend = Aer.get_backend('qasm_simulator')
    measurements = []
    for q in range(n):
        if bases[q] == 0: # measuring in Z-basis
            message[q].measure(0,0)
        if bases[q] == 1: # measuring in X-basis
            message[q].h(0)
            message[q].measure(0,0)
        result = execute(message[q], backend, shots=1, memory=True).result()
        measured_bit = int(result.get_memory()[0])
        measurements.append(measured_bit)
    return measurements

In [75]:
## Step 3
# Decide which basis to measure in:
bob_bases = randint(2, size=n)
bob_results = measure_message(message, bob_bases)
print(bob_bases)
print("Bob's first chosen basis = %i" % bob_bases[0])

[1 0 0 0 1 0 1 1 1 1 1 0 0 1 0 1 1 0 0 1 0 1 0 0 0 0 1 1 0 1 0 0 0 1 0 0 0
 1 1 1 0 0 0 1 0 1 0 0 0 1 0 1 1 1 0 0 0 1 1 0 0 0 0 0 0 1 0 1 0 1 1 1 1 0
 0 0 0 1 0 0 1 0 1 0 0 1 0 0 1 1 1 0 0 1 1 1 1 1 1 0]
Bob's first chosen basis = 1


In [76]:
list_index = []
def remove_garbage(a_bases, b_bases, bits):
    good_bits = []
    for q in range(n):
        if a_bases[q] == b_bases[q]:
            # If both used the same basis, add
            # this to the list of 'good' bits
            good_bits.append(bits[q])
            list_index.append(q)
    return good_bits

In [77]:
## Step 4
# Remove bits where Alice's chosen bases are not equal to that of Bob's
alice_key = remove_garbage(alice_bases, bob_bases, alice_bits)

Eve_remove = []
for i in list_index:
    Eve_remove.append(intercepted_message[i])

bob_key = remove_garbage(alice_bases, bob_bases, bob_results)

ab_same = 0
# Compare the bits between Alice's and Bob's
for i in range(len(alice_key)):
    if (alice_key[i] == bob_key[i]): ab_same += 1
        
# Compare the bits between Alice's and Eve's
ae_same = 0
for i in range(len(alice_key)):
    if (alice_key[i] == Eve_remove[i]): ae_same += 1

ab_same_pb = (ab_same/len(alice_key))*100
ae_same_pb = (ae_same / len(alice_key))*100
print("Alice's key: ", alice_key)
print("Bob's key: ", bob_key)
print("The probability that Alice's = Bob's bits after removing: %2.f%%"%ab_same_pb)
print("The probability that Alice's = Eve's bits after removing: %2.f%%"%ae_same_pb)

Alice's key:  [0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1]
Bob's key:  [0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1]
The probability that Alice's = Bob's bits after removing: 79%
The probability that Alice's = Eve's bits after removing: 69%


In [78]:
# Problem 3-(a)
p = (ab_same_pb*ae_same_pb)/100
print("Probability of Eve guessing Alice’s bit correctly = %2.f%%" % p)

Probability of Eve guessing Alice’s bit correctly = 55%


In [79]:
# Problem 3-(b)

In [80]:
n = 100; # number of qubits used in the BB84
np.random.seed(seed=3)
## Step 1
# Alice generates bits
alice_bits = randint(2, size=n)
print(alice_bits)
print("Alice's first bit = %i" % alice_bits[0])

[0 0 1 1 0 0 0 1 1 1 0 1 1 1 0 1 1 0 0 0 0 1 1 0 0 0 1 0 0 0 0 1 0 1 1 0 1
 0 0 1 1 0 0 1 0 1 0 1 1 1 1 0 1 0 0 1 1 1 0 0 0 1 0 0 0 1 0 0 1 1 0 0 1 1
 1 0 1 1 1 1 1 1 0 1 0 0 1 1 0 1 0 0 0 0 0 1 1 0 1 1]
Alice's first bit = 0


In [81]:
def encode_message(bits, bases):
    message = []
    for i in range(n):
        qc = QuantumCircuit(1,1)
        if bases[i] == 0: # Prepare qubit in X-basis
            if bits[i] == 0:
                pass 
            else:
                qc.x(0)
        else: # Prepare qubit in Z-basis
            if bits[i] == 0:
                qc.h(0)
            else:
                qc.x(0)
                qc.h(0)
        qc.barrier()
        message.append(qc)
    return message

In [82]:
## Step 2
# Create an array to tell us which qubits are encoded in which bases
alice_bases = randint(2, size=n)
message = encode_message(alice_bits, alice_bases)
print(alice_bases)
print("Alice's first chosen bit = %i" % alice_bits[0])
print("Alice's first chosen basis = %i" % alice_bases[0])

[1 0 0 1 1 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1 1 1 0 0 1 1 0 0 1 1 1 1 0 0
 1 1 0 0 0 0 0 0 0 0 1 1 0 0 0 1 0 1 1 1 0 0 1 0 1 0 1 1 0 0 1 0 0 1 1 1 1
 1 0 0 0 0 0 1 1 1 0 0 0 0 0 0 1 0 1 0 0 0 1 0 1 1 1]
Alice's first chosen bit = 0
Alice's first chosen basis = 1


In [83]:
def intercept_message(message, bases):
    backend = Aer.get_backend('qasm_simulator')
    measurements = []
    for q in range(n):
        if bases[q] == 0: # measuring in Ry-basis
            message[q].u3(pi/4, 0, pi, 0).inverse()
            message[q].measure(0,0)
        if bases[q] == 1: # measuring in Ry-basis
            message[q].u3(pi/4, 0, pi, 0).inverse()
            message[q].measure(0,0)
        result = execute(message[q], backend, shots=1, memory=True).result()
        measured_bit = int(result.get_memory()[0])
        measurements.append(measured_bit)
    return measurements

In [84]:
## Interception!!
eve_bases = randint(2, size=n)
intercepted_message = intercept_message(message, eve_bases)
print(eve_bases)
print("Eve's first chosen basis = %i\n" % alice_bases[0])
print(intercepted_message)
print("Eve's first intercepted message = %i" % intercepted_message[0])

[0 1 1 0 1 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 1 0 1 0 1 0 1 0 1 0 1 1 0 1
 0 1 0 1 0 1 1 1 1 1 0 1 0 1 1 1 1 0 0 0 1 0 1 0 0 1 0 1 0 0 0 1 0 0 1 1 1
 0 1 0 1 0 1 1 0 1 1 0 1 1 0 0 0 1 1 1 0 1 0 0 0 1 0]
Eve's first chosen basis = 1

[0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
Eve's first intercepted message = 0


In [85]:
def measure_message(message, bases):
    backend = Aer.get_backend('qasm_simulator')
    measurements = []
    for q in range(n):
        if bases[q] == 0: # measuring in Z-basis
            message[q].measure(0,0)
        if bases[q] == 1: # measuring in X-basis
            message[q].h(0)
            message[q].measure(0,0)
        result = execute(message[q], backend, shots=1, memory=True).result()
        measured_bit = int(result.get_memory()[0])
        measurements.append(measured_bit)
    return measurements

In [86]:
## Step 3
# Decide which basis to measure in:
bob_bases = randint(2, size=n)
bob_results = measure_message(message, bob_bases)
print(bob_bases)
print("Bob's first chosen basis = %i" % bob_bases[0])

[1 0 0 0 1 0 1 1 1 1 1 0 0 1 0 1 1 0 0 1 0 1 0 0 0 0 1 1 0 1 0 0 0 1 0 0 0
 1 1 1 0 0 0 1 0 1 0 0 0 1 0 1 1 1 0 0 0 1 1 0 0 0 0 0 0 1 0 1 0 1 1 1 1 0
 0 0 0 1 0 0 1 0 1 0 0 1 0 0 1 1 1 0 0 1 1 1 1 1 1 0]
Bob's first chosen basis = 1


In [87]:
list_index = []
def remove_garbage(a_bases, b_bases, bits):
    good_bits = []
    for q in range(n):
        if a_bases[q] == b_bases[q]:
            # If both used the same basis, add
            # this to the list of 'good' bits
            good_bits.append(bits[q])
            list_index.append(q)
    return good_bits

In [88]:
## Step 4
# Remove bits where Alice's chosen bases are not equal to that of Bob's
alice_key = remove_garbage(alice_bases, bob_bases, alice_bits)

Eve_remove = []
for i in list_index:
    Eve_remove.append(intercepted_message[i])

bob_key = remove_garbage(alice_bases, bob_bases, bob_results)

ab_same = 0
# Compare the bits between Alice's and Bob's
for i in range(len(alice_key)):
    if (alice_key[i] == bob_key[i]): ab_same += 1
        
# Compare the bits between Alice's and Eve's
ae_same = 0
for i in range(len(alice_key)):
    if (alice_key[i] == Eve_remove[i]): ae_same += 1

ab_same_pb = (ab_same / len(alice_key))*100
ae_same_pb = (ae_same / len(alice_key))*100
print("Alice's key: ", alice_key)
print("Bob's key: ", bob_key)
print("The probability that Alice's = Bob's bits after removing: %2.f%%"%ab_same_pb)
print("The probability that Alice's = Eve's bits after removing: %2.f%%"%ae_same_pb)

Alice's key:  [0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1]
Bob's key:  [1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1]
The probability that Alice's = Bob's bits after removing: 73%
The probability that Alice's = Eve's bits after removing: 83%


In [89]:
# Problem 3-(b)
p = (ab_same_pb*ae_same_pb)/100
print("Probability of Eve guessing Alice’s bit correctly = %2.f%%" % p)

Probability of Eve guessing Alice’s bit correctly = 60%
